In [1]:
import os

In [2]:
%pwd

'/Users/rahulshelke/Documents/Data-Science/Data-Science-Projects/finance_complaint/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/rahulshelke/Documents/Data-Science/Data-Science-Projects/finance_complaint'

## config_entity.py

In [5]:
from dataclasses import dataclass
from finance_complaint.entity.config_entity import TrainingPipelineConfig
from finance_complaint.exception import FinanceException
from finance_complaint.logger import logging as logger
import os, sys

Reading environment variables
Read Complete!


In [6]:
# Model Trainer Constants
MODEL_TRAINER_BASE_ACCURACY = 0.6
MODEL_TRAINER_DIR = "model_trainer"
MODEL_TRAINER_TRAINED_MODEL_DIR = "trained_model"
MODEL_TRAINER_MODEL_NAME = "finance_estimator"
MODEL_TRAINER_LABEL_INDEXER_DIR = "label_indexer"
MODEL_TRAINER_MODEL_METRIC_NAMES = [
    'f1',
    "weightedPrecision",
    "weightedRecall",
    "weightedTruePositiveRate",
    "weightedFalsePositiveRate",
    "weightedFMeasure",
    "truePositiveRateByLabel",
    "falsePositiveRateByLabel",
    "precisionByLabel",
    "recallByLabel",
    "fMeasureByLabel"
]

In [7]:
@dataclass
class ModelTrainerConfig:

    def __init__(self, training_pipeline_config: TrainingPipelineConfig) -> None:
        try:
            model_trained_dir = os.path.join(training_pipeline_config.artifact_dir, MODEL_TRAINER_DIR)
            self.trained_model_file_path = os.path.join(model_trained_dir, MODEL_TRAINER_TRAINED_MODEL_DIR, MODEL_TRAINER_MODEL_NAME)
            self.label_indexer_model_dir = os.path.join(model_trained_dir, MODEL_TRAINER_LABEL_INDEXER_DIR)
            self.base_accuracy = MODEL_TRAINER_BASE_ACCURACY
            self.metric_list = MODEL_TRAINER_MODEL_METRIC_NAMES
        except Exception as e:
            raise FinanceException(e, sys)

## artifact_entity.py

In [8]:
@dataclass
class PartialModelTrainerMetricArtifact:
    f1_score: float
    precision_score: float
    recall_score: float

    def _asdict(self):
        return self.__dict__
    
@dataclass
class PartialModelTrainerRefArtifact:
    trained_model_file_path: str
    label_indexer_model_file_path: str

    def _asdict(self):
        return self.__dict__
    

class ModelTrainerArtifact:

    def __init__(self, 
                 model_trainer_ref_artifact: PartialModelTrainerRefArtifact,
                 model_trainer_train_metric_artifact: PartialModelTrainerMetricArtifact,
                 model_trainer_test_metric_artifact: PartialModelTrainerMetricArtifact
                 ):
        self.model_trainer_ref_artifact = model_trainer_ref_artifact
        self.model_trainer_metric_artifact = model_trainer_train_metric_artifact
        self.model_test_metric_artifact = model_trainer_test_metric_artifact

    @staticmethod
    def construct_object(**kwargs):
        model_trainer_ref_artifact = PartialModelTrainerRefArtifact(**(kwargs['model_trainer_ref_artifact']))
        model_trainer_train_metric_artifact = PartialModelTrainerMetricArtifact(**(kwargs['model_trainer_train_metric_artifact']))
        model_trainer_test_metric_artifact = PartialModelTrainerMetricArtifact(**(kwargs['model_trainer_test_metric_artifact']))

        model_trainer_artifact = ModelTrainerArtifact(
            model_trainer_ref_artifact,
            model_trainer_train_metric_artifact,
            model_trainer_test_metric_artifact
        )

        return model_trainer_artifact
    
    def _asdict(self):
        try:
            response = dict()
            response['model_trainer_ref_artifact'] = self.model_trainer_ref_artifact._asdict()
            response['model_trainer_train_metric_artifact'] = self.model_trainer_metric_artifact._asdict()
            response['model_trainer_test_metric_artifact'] = self.model_test_metric_artifact._asdict()

            return response
        except Exception as e:
            raise e

## model_trainer.py

In [ ]:
import os, sys
from typing import List
from pyspark.sql import DataFrame

from finance_complaint.entity.schema import FinanceDataSchema
from finance_complaint.exception import FinanceException
from finance_complaint.logger import logging as logger

from finance_complaint.entity.artifact_entity import (DataTransformationArtifact, 
#                                                       PartialModelTrainerRefArtifact, 
#                                                       PartialModelTrainerMetricArtifact, 
#                                                       ModelTrainerArtifact
)

# from finance_complaint.entity.config_entity import ModelTrainerConfig

from finance_complaint.config.spark_manager import spark_session
from finance_complaint.utils import get_score

from pyspark.ml.feature import StringIndexer, StringIndexerModel
from pyspark.ml.feature import IndexToString
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.pipeline import Pipeline, PipelineModel


MongoClient(host=['ac-zciqt62-shard-00-01.uhbxanv.mongodb.net:27017', 'ac-zciqt62-shard-00-02.uhbxanv.mongodb.net:27017', 'ac-zciqt62-shard-00-00.uhbxanv.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, retrywrites=True, w='majority', appname='mlcluster', authsource='admin', replicaset='atlas-10vcjq-shard-0', tls=True, server_api=<pymongo.server_api.ServerApi object at 0x12f0401a0>)


25/05/10 18:52:34 WARN Utils: Your hostname, Rahuls-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.1.5 instead (on interface en0)
25/05/10 18:52:34 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/10 18:52:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/05/10 18:52:36 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 53082)
Traceback (most recent call last):
  File "/Users/rahulshelke/Documents/Data-Science/Data-Science-Projects/finance_complaint/venv/lib/python3.13/socketserver.py", line 318, in _handle_request_noblock
    self.process_request(request, client_address)
    ~~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/rahulshelke/Documents/Data-Science/Data-Science-Projects/finance_complaint/venv/lib/python3.13/socketserver.py", line 349, in process_request
    self.finish_request(request, client_address)
    ~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/rahulshelke/Documents/Data-Science/Data-Science-Projects/finance_complaint/venv/lib/python3.13/socketserver.py", line 362, in finish_request
    self.RequestHandlerClass(request, client_address, self)
    ~~~~~~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/rahulshelke/Documents/Data-Science/Data

In [ ]:
class ModelTrainer:

    def __init__(self,
                 data_transformation_artifact: DataTransformationArtifact,
                 model_trainer_config: ModelTrainerConfig,
                 schema = FinanceDataSchema()):
        self.data_transformation_artifact = data_transformation_artifact
        self.model_trainer_config = model_trainer_config
        self.schema = schema

    def get_train_test_dataframe(self) -> List[DataFrame]:
        try:
            train_file_path = self.data_transformation_artifact.transformed_train_file_path
            test_file_path = self.data_transformation_artifact.transformed_test_file_path
            train_dataframe: DataFrame = spark_session.read.parquet(train_file_path)
            test_dataframe: DataFrame = spark_session.read.parquet(test_file_path)
            print(f"Train row: {train_dataframe.count()} Test row: {test_dataframe.count()}")
            dataframes: List[DataFrame] = [train_dataframe, test_dataframe]

            return dataframes
        except Exception as e:
            raise FinanceException(e, sys)
        
    def get_scores(self, dataframe: DataFrame, metric_names: List[str]) -> List[tuple]:
        try:
            if metric_names is None:
                metric_names = self.model_trainer_config.metric_list

            scores : List[tuple] = []
            for metric_name in metric_names:
                score = get_score(metric_name=metric_name,
                                  dataframe=dataframe,
                                  label_col = self.schema.target_indexed_label,
                                  prediction_col=self.schema.prediction_column_name,)
                scores.append((metric_name, score))
            return scores
        except Exception as e:
            raise FinanceException(e, sys)
        
    def get_model(self, label_indexer_model: StringIndexerModel) -> Pipeline:
        try:
            stages = []
            logger.info("Creating Random Forest Classifier class.")
            random_forest_clf = RandomForestClassifier(labelCol=self.schema.target_indexed_label,
                                                       featuresCol=self.schema.scaled_vector_input_features)
            logger.info("Creating Label generator.")
            label_generator = IndexToString(inputCol=self.schema.prediction_column_name,
                                            outputCol=f"{self.schema.prediction_column_name}_{self.schema.target_column}",
                                            labels=label_indexer_model.labels)
            stages.append(random_forest_clf)
            stages.append(label_generator)
            pipeline = Pipeline(stages=stages)
            return pipeline
        except Exception as e:
            raise FinanceException(e, sys)
        
    def export_trained_model(self,model: PipelineModel) -> PartialModelTrainerRefArtifact:
        try:
            transformed_pipeline_file_path=  self.data_transformation_artifact.exported_pipeline_file_path
            transformed_pipeline = PipelineModel.load(transformed_pipeline_file_path)

            updated_stages = transformed_pipeline.stages + model.stages
            transformed_pipeline.stages = updated_stages

            logger.info("Creating trained model directory")
            trained_model_file_path = self.model_trainer_config.trained_model_file_path
            os.makedirs(os.path.dirname(trained_model_file_path), exist_ok=True)

            transformed_pipeline.save(trained_model_file_path)

            ref_artifact = PartialModelTrainerRefArtifact(
                trained_model_file_path=trained_model_file_path,
                label_indexer_model_file_path=self.model_trainer_config.label_indexer_model_dir
            )

            logger.info(f"Model trainer reference artifact: {ref_artifact}")

            return ref_artifact
        except Exception as e:
            raise FinanceException(e, sys)
        
    def initiate_model_training(self) -> ModelTrainerArtifact:
        try:
            dataframes = self.get_train_test_dataframe()
            train_dataframe, test_dataframe = dataframes[0], dataframes[1]

            print(f"Train row: {train_dataframe.count()} Test row: {test_dataframe.count()}")

            label_indexer = StringIndexer(inputCol=self.schema.target_column,
                          outputCol=self.schema.target_indexed_label)
            
            label_indexer_model = label_indexer.fit(train_dataframe)

            os.makedirs(os.path.dirname(self.model_trainer_config.label_indexer_model_dir), exist_ok=True)
            label_indexer_model.save(self.model_trainer_config.label_indexer_model_dir)

            train_dataframe = label_indexer_model.transform(train_dataframe)
            test_dataframe = label_indexer_model.transform(test_dataframe)

            model = self.get_model(label_indexer_model=label_indexer_model)
            trained_model = model.fit(train_dataframe)

            train_dataframe_pred = trained_model.transform(train_dataframe)
            test_dataframe_pred = trained_model.transform(test_dataframe)

            print(f"number of row in trauning: {train_dataframe_pred.count()}")
            scores = self.get_scores(dataframe=train_dataframe_pred, metric_names=self.model_trainer_config.metric_list)

            train_metric_artifact = PartialModelTrainerMetricArtifact(f1_score=scores[0][1],
                                                                      precision_score=scores[1][1],
                                                                      recall_score=scores[2][1])
            logger.info(f"Model trainer train metric: {train_metric_artifact}")

            print(f"number of rows in testing: {test_dataframe_pred.count()}")
            scores = self.get_scores(dataframe=test_dataframe_pred,
                                    metric_names=self.model_trainer_config.metric_list)
            test_metric_artifact = PartialModelTrainerMetricArtifact(f1_score=scores[0][1],
                                                                     precision_score=scores[1][1],
                                                                     recall_score=scores[2][1])
            logger.info(f"Model trainer test metric: {test_metric_artifact}")
            ref_artifact = self.export_trained_model(model=trained_model)
            model_trainer_artifact = ModelTrainerArtifact(model_trainer_ref_artifact=ref_artifact,
                                                          model_trainer_train_metric_artifact=train_metric_artifact,
                                                          model_trainer_test_metric_artifact=test_metric_artifact)
            
            logger.info(f"Model trainer artifact: {model_trainer_artifact}")

            return model_trainer_artifact
        
        except Exception as e:
            raise FinanceException(e, sys)
        

## training.py

In [11]:
from finance_complaint.exception import FinanceException
from finance_complaint.logger import logging as logger
from finance_complaint.entity.schema import FinanceDataSchema

from finance_complaint.entity.config_entity import (DataIngestionConfig, 
                                                    DataValidationConfig,
                                                    DataTransformationConfig)
from finance_complaint.entity.artifact_entity import (DataIngestionArtifact,
                                                      DataValidationArtifact,
                                                      DataTransformationArtifact)

from finance_complaint.components.data_ingestion import DataIngestion
from finance_complaint.components.data_validation import DataValidation
from finance_complaint.components.data_transformation import DataTransformation

import sys, os


In [12]:
class TrainingPipeline:
    """ 
    this is a training pipeline comprising each components from flow chart
    """
    def __init__(self, training_pipeline_config: TrainingPipelineConfig):
        self.training_pipeline_config: TrainingPipelineConfig = training_pipeline_config

    def start_data_ingestion(self) -> DataIngestionArtifact:
        """
        responsible for starting data ingestion 
        """
        try:
            data_ingestion_config = DataIngestionConfig(training_pipeline_config=self.training_pipeline_config)
            data_ingestion = DataIngestion(data_ingestion_config=data_ingestion_config)
            data_ingestion_artifact = data_ingestion.initiate_data_ingestion()
            return data_ingestion_artifact
        
        except Exception as e:
            logger.debug(f"Error: {e}")
            raise FinanceException(e, sys)
        
    def start_data_validation(self, data_ingestion_artifact: DataIngestionArtifact) -> DataValidationArtifact:
        """
        takes ingested data artifact and validate that data by generating validation artifact
        """
        try:
            data_validation_config = DataValidationConfig(training_pipeline_config=self.training_pipeline_config)
            data_validation = DataValidation(
                data_validation_config=data_validation_config,
                data_ingestion_artifact=data_ingestion_artifact
            )
            data_validation_artifact = data_validation.initiate_data_validation()
            return data_validation_artifact
        except Exception as e:
            raise FinanceException(e, sys)
        
    def start_data_transformation(self, data_validation_artifact: DataValidationArtifact) -> DataTransformationArtifact:
        """ 
        responsible for data transformation 
        """
        try:
            data_transformation_config = DataTransformationConfig(training_pipeline_config=self.training_pipeline_config)
            data_transformation = DataTransformation(data_validation_artifact=data_validation_artifact,
                                                     data_transformation_config=data_transformation_config)
            
            data_transformation_artifacts = data_transformation.initiate_data_transformation()
            return data_transformation_artifacts
        except Exception as e:
            raise FinanceException(e, sys)
        
    def start_model_trainer(self, data_transformation_artifact: DataTransformationArtifact) -> ModelTrainerArtifact:
        try:
            model_trainer_config = ModelTrainerConfig(training_pipeline_config=self.training_pipeline_config)
            model_trainer = ModelTrainer(data_transformation_artifact=data_transformation_artifact,
                                         model_trainer_config=model_trainer_config)
            model_trainer_artifact = model_trainer.initiate_model_training()

            return model_trainer_artifact
        except Exception as e:
            raise FinanceException(e, sys)
        
    def start(self):
        try:
            # initalizating data ingestion
            data_ingestion_artifact = self.start_data_ingestion()
            # initalizing data validation
            data_validation_artifact = self.start_data_validation(data_ingestion_artifact=data_ingestion_artifact)
            # initalizing data transformation
            data_transformation_artifact = self.start_data_transformation(data_validation_artifact=data_validation_artifact)
            # initalizing model training
            model_trainer_artifact = self.start_model_trainer(data_transformation_artifact=data_transformation_artifact)

        except Exception as e:
            raise FinanceException(e, sys)    


## train.py

In [13]:
from finance_complaint.pipeline.training import TrainingPipeline
# from finance_complaint.entity.config_entity import TrainingPipelineConfig

if __name__ == "__main__":
    training_pipeline_config = TrainingPipelineConfig()
    training_pipeline = TrainingPipeline(training_pipeline_config=training_pipeline_config)
    training_pipeline.start()

Row: [10000] Column: [12]
Expected Column: ['consumer_disputed', 'company_response', 'consumer_consent_provided', 'submitted_via', 'issue', 'date_sent_to_company', 'date_received']
Present Columns: ['company', 'company_response', 'consumer_consent_provided', 'consumer_disputed', 'date_received', 'date_sent_to_company', 'issue', 'product', 'state', 'submitted_via', 'timely', 'zip_code']
root
 |-- company: string (nullable = true)
 |-- company_response: string (nullable = true)
 |-- consumer_consent_provided: string (nullable = true)
 |-- consumer_disputed: string (nullable = true)
 |-- date_received: string (nullable = true)
 |-- date_sent_to_company: string (nullable = true)
 |-- issue: string (nullable = true)
 |-- product: string (nullable = true)
 |-- state: string (nullable = true)
 |-- submitted_via: string (nullable = true)
 |-- timely: string (nullable = true)
 |-- zip_code: string (nullable = true)

{'inputCols': ['company_response', 'consumer_consent_provided', 'submitted_via'

{'inputCols': ['company_response', 'consumer_consent_provided', 'submitted_via'], 'outputCols': ['im_company_response', 'im_consumer_consent_provided', 'im_submitted_via']}


6960 2


3040 2
Train row: 6960 Test row: 3040
Train row: 6960 Test row: 3040
number of row in trauning: 6960
f1 score: 0.7152862159644536
weightedPrecision score: 0.6448366899524375
weightedRecall score: 0.8030172413793103
weightedTruePositiveRate score: 0.8030172413793103
weightedFalsePositiveRate score: 0.8030172413793103
weightedFMeasure score: 0.7152862159644536
truePositiveRateByLabel score: 1.0
falsePositiveRateByLabel score: 1.0
precisionByLabel score: 0.8030172413793103
recallByLabel score: 1.0
fMeasureByLabel score: 0.8907482667941667
number of rows in testing: 3040
f1 score: 0.725703794369645
weightedPrecision score: 0.6569529085872575
weightedRecall score: 0.8105263157894737
weightedTruePositiveRate score: 0.8105263157894737
weightedFalsePositiveRate score: 0.8105263157894737
weightedFMeasure score: 0.725703794369645
truePositiveRateByLabel score: 1.0
falsePositiveRateByLabel score: 1.0
precisionByLabel score: 0.8105263157894737
recallByLabel score: 1.0
fMeasureByLabel score: 0.8953

25/05/10 18:53:50 ERROR Executor: Exception in task 0.0 in stage 383.0 (TID 516)
org.apache.spark.SparkException: [FAILED_EXECUTE_UDF] Failed to execute user defined function (`StringIndexerModel$$Lambda$4343/435456080`: (string) => double).
	at org.apache.spark.sql.errors.QueryExecutionErrors$.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala:198)
	at org.apache.spark.sql.errors.QueryExecutionErrors.failedExecuteUserDefinedFunctionError(QueryExecutionErrors.scala)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenEvaluatorFactory$WholeStageCodegenPartitionEvaluator$$anon$1.hasNext(WholeStageCodegenEvaluatorFactory.scala:43)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:460)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:460)
	

FinanceException: Error occured in script: [ /Users/rahulshelke/Documents/Data-Science/Data-Science-Projects/finance_complaint/finance_complaint/pipeline/training.py ] at line number: [ 110 ] error message: [Error occured in script: [ /Users/rahulshelke/Documents/Data-Science/Data-Science-Projects/finance_complaint/finance_complaint/pipeline/training.py ] at line number: [ 83 ] error message: [Error occured in script: [ /Users/rahulshelke/Documents/Data-Science/Data-Science-Projects/finance_complaint/finance_complaint/components/model_evalutaion.py ] at line number: [ 115 ] error message: [Error occured in script: [ /Users/rahulshelke/Documents/Data-Science/Data-Science-Projects/finance_complaint/finance_complaint/components/model_evalutaion.py ] at line number: [ 87 ] error message: [Error occured in script: [ /Users/rahulshelke/Documents/Data-Science/Data-Science-Projects/finance_complaint/finance_complaint/utils.py ] at line number: [ 42 ] error message: [An error occurred while cal